In [1]:
%pip install requests
%pip install beautifulsoup4
%pip install pandas


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\305ki\.pyenv\pyenv-win\versions\3.10.4\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\305ki\.pyenv\pyenv-win\versions\3.10.4\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\305ki\.pyenv\pyenv-win\versions\3.10.4\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
circuit_data = []

url = f"https://pitwall.app/circuits"
print("Fetching URL:", url)
response = requests.get(url)

html_data = response.content

soup = BeautifulSoup(html_data, 'html.parser')

circuit_table = soup.find('table', class_='data-table')

headers = [header.get_text(strip=True) for header in circuit_table.find_all('th')]
headers.append('Circuit URL')

print("Headers:", headers)



Fetching URL: https://pitwall.app/circuits
Headers: ['Circuit', 'Location', 'Last race', 'Races', 'Circuit URL']


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Extract the processing logic into a function
def extract_li_value(soup_obj, key_text):
    strong_tag = soup_obj.find('strong', string=lambda text: text and key_text in text)
    if strong_tag:
        full_text = strong_tag.parent.get_text(strip=True)
        return full_text.split(':', 1)[1].strip()
    return None

def process_circuit_row(row):
    """Process a single circuit row - fetch details and return data dict"""
    circuit_url_full = None
    try:
        cols = row.find_all('td')
        cols_text = [col.get_text(strip=True) for col in cols]
        circuit_url = cols[0].find('a')['href']
        cols_text.append(circuit_url)
        row_dict = dict(zip(headers, cols_text))

        circuit_url_full = "https://pitwall.app" + circuit_url
        print(f"Fetching Circuit URL: {circuit_url_full}")

        response = requests.get(circuit_url_full)
        html_data = response.content
        soup = BeautifulSoup(html_data, 'html.parser')

        key = soup.find('div', class_='title', string='Circuit type').get_text(strip=True)
        value = soup.find('div', class_='title', string='Circuit type').find_next_sibling('div', class_='content').get_text(strip=True)

        row_dict[key] = value

        circuit_stats_ul = soup.find('section', id='circuit-stats').find('ul')

        row_dict['First Grand Prix'] = extract_li_value(circuit_stats_ul, 'First Grand Prix:')
        row_dict['Track length'] = extract_li_value(circuit_stats_ul, 'Track length:')
        row_dict['Turns'] = extract_li_value(circuit_stats_ul, 'Turns:')
        row_dict['Direction'] = extract_li_value(circuit_stats_ul, 'Direction:')
        row_dict['Lap record'] = extract_li_value(circuit_stats_ul, 'Lap record:')

        return row_dict
    except Exception as e:
        url_str = circuit_url_full if circuit_url_full else "unknown URL"
        print(f"Error processing {url_str}: {e}")
        return None

# Collect all rows first
rows = list(circuit_table.find('tbody').find_all('tr'))
total = len(rows)
print(f"Processing {total} circuits with multithreading...")

# Use ThreadPoolExecutor for parallel processing
# Adjust max_workers based on your needs (10-20 is usually good for I/O-bound tasks)
max_workers = 20
circuit_data = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_row = {executor.submit(process_circuit_row, row): row for row in rows}
    
    # Process completed tasks as they finish
    completed = 0
    for future in as_completed(future_to_row):
        completed += 1
        result = future.result()
        if result:
            circuit_data.append(result)
        
        # Progress update every 10 circuits
        if completed % 10 == 0 or completed == total:
            elapsed = time.time() - start_time
            remaining = ((elapsed / completed) * (total - completed)) if completed > 0 else 0
            print(f"Progress: {completed}/{total} circuits processed. "
                  f"Elapsed: {elapsed:.1f}s, Estimated remaining: {remaining:.1f}s")

elapsed_total = time.time() - start_time
print(f"\nCompleted! Processed {len(circuit_data)} circuits in {elapsed_total:.1f} seconds")
print(f"Average: {elapsed_total/len(circuit_data):.2f} seconds per circuit")



Processing 77 circuits with multithreading...
Fetching Circuit URL: https://pitwall.app/circuits/adelaide-street-circuit
Fetching Circuit URL: https://pitwall.app/circuits/ain-diab-circuit
Fetching Circuit URL: https://pitwall.app/circuits/aintree-motor-racing-circuit
Fetching Circuit URL: https://pitwall.app/circuits/algarve-international-circuit
Fetching Circuit URL: https://pitwall.app/circuits/anderstorp-raceway
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-do-estoril
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-hermanos-rodriguez
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-internacional-nelson-piquet
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-internazionale-del-mugello
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-internazionale-enzo-e-dino-ferrari
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-jose-carlos-pace
Fetching Circuit URL: https://pitwall.app/circuits/autodromo-juan-y-oscar-gal

In [5]:
def to_seconds(time_str):
    try:
        minutes, seconds_ms = time_str.split(':')
        return float(minutes) * 60 + float(seconds_ms)
    except:
        return None


In [6]:
circuit_df = pd.DataFrame(circuit_data)
circuit_df[['City', 'Country']] = circuit_df['Location'].str.split(',', expand=True)
circuit_df.drop(columns=['Location'], inplace=True)
circuit_df = circuit_df.rename(columns={'Last race': 'Last Grand Prix'})
columns_order = ['Circuit', 'City', 'Country', 'Track length', 'Turns', 'Direction', 'Circuit type', 'First Grand Prix', 'Last Grand Prix', 'Races', 'Circuit URL', 'Lap record', ]
circuit_df = circuit_df[columns_order]

circuit_df[['Best Lap Timing', 'Driver/Year']] = circuit_df['Lap record'].str.extract(r'(\d+:\d+\.\d+)\s*\((.+)\)')
circuit_df[['Best Lap Driver', 'Best Lap Year']] = circuit_df['Driver/Year'].str.split(',\s*', expand=True)

circuit_df['Best Lap Year'] = circuit_df['Best Lap Year'].str.replace(r'[()]', '', regex=True)
circuit_df['Best Lap Time'] = circuit_df['Best Lap Timing'].apply(to_seconds)

circuit_df['Track length'] = circuit_df['Track length'].str.replace(' km', '').astype(float)
circuit_df = circuit_df.rename(columns={'Track length': 'Track Length (km)' ,'Circuit type': 'Circuit Type'})

circuit_df = circuit_df.drop(columns=['Lap record', 'Driver/Year'])
circuit_df.to_csv('circuit_url.csv', index=False)
circuit_df = circuit_df.drop(columns=['Circuit URL'])
circuit_df.to_csv('circuit.csv', index=False)